In [1]:
import numpy as np
import tensorflow_hub as hub
import tensorflow_text 
import torch
from transformers import BertTokenizer, BertModel
import re
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tqdm

In [2]:
# embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

df = pd.read_csv('puck.csv')
df['tag'].value_counts()




IT-компании                       4388
Разработка веб-сайтов *           2145
Информационная безопасность *     2029
Настройка Linux *                 1730
Компьютерное железо               1545
Гаджеты                           1371
Социальные сети и сообщества      1177
Name: tag, dtype: int64

In [3]:
# df = df[df['brave_new_text'] != 'removed by selfcensorship ']
# df = df[df['brave_new_text'] != 'сообщение удалено ']
# df = df[df['brave_new_text'] != 'nan']
# df.reset_index(drop=True)


In [4]:
msg_train, msg_test, y_train, y_test = train_test_split(df.brave_new_textpunc_stopwords, df.tag,random_state=1)

In [5]:
# df['brave_new_text'][18]

In [6]:

# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# model = BertModel.from_pretrained('bert-base-multilingual-cased',
#                                   output_hidden_states = True, # Whether the model returns all hidden-states.
#                                   )
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")

def embed(text):
    # Load pre-trained model tokenizer (vocabulary)
    
    # Add the special tokens.
    marked_text = "[CLS] " + str(text)  + " [SEP]"
    # Split the sentence into tokens.
#     tokenized_text = tokenizer.tokenize(marked_text)
    tokenized_text = tokenizer.encode(marked_text,padding=True, truncation=True,max_length=512)
    # Map the token strings to their vocabulary indeces.
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
#     # Mark each of the 22 tokens as belonging to sentence "1".
#     segments_ids = [1] * len(tokenized_text)
#     # Convert inputs to PyTorch tensors
#     tokens_tensor = torch.tensor([indexed_tokens])
#     segments_tensors = torch.tensor([segments_ids])
    # Load pre-trained model (weights)
    # # Run the text through BERT, and collect all of the hidden states produced
    # # from all 12 layers. 
    with torch.no_grad():
#         outputs = model(tokens_tensor, segments_tensors)
        outputs=model(tokenized_text)
        hidden_states = outputs[2]
        # Each layer in the list is a torch tensor.
    # Concatenate the tensors for all layers. We use `stack` here to
    # create a new dimension in the tensor.
    token_embeddings = torch.stack(hidden_states, dim=0)
    # Remove dimension 1, the "batches".
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    # Swap dimensions 0 and 1.
    token_embeddings = token_embeddings.permute(1,0,2)
    # `hidden_states` has shape [13 x 1 x 22 x 768]
    # `token_vecs` is a tensor with shape [22 x 768]
    token_vecs = hidden_states[-2][0]
    # Calculate the average of all 22 token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)
    # Stores the token vectors, with shape [22 x 768]
#     token_vecs_sum = []
#     # `token_embeddings` is a [22 x 12 x 768] tensor.
#     # For each token in the sentence...
#     for token in token_embeddings:
#         # `token` is a [12 x 768] tensor
#         # Sum the vectors from the last four layers.
#         sum_vec = torch.sum(token[-4:], dim=0)
#         # Use `sum_vec` to represent `token`.
#         token_vecs_sum.append(sum_vec)
# #     print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))
# #     sentence_vector = np.mean(token_vecs_sum,axis=0)
#     print(token_vecs.size())
#     print(sentence_embedding.size())
    return sentence_embedding
    
        
    
    

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, max_length=512,return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
        embeddings = model_output.last_hidden_state[:, 0, :]
        embeddings = torch.nn.functional.normalize(embeddings)
        return embeddings[0].cpu().numpy()
    
print(embed_bert_cls('привет мир', model, tokenizer))

[-2.84503493e-02  2.48572677e-02  4.52326089e-02 -7.95979351e-02
 -1.53341526e-02  1.95008032e-02  6.84027299e-02 -2.58912588e-03
 -8.90688151e-02 -7.47816935e-02  5.33769056e-02  3.95353697e-02
  2.15761587e-02  1.92577783e-02  8.65387917e-03  1.72012728e-02
  5.63668720e-02  1.71543378e-02  2.38393713e-02  1.00879058e-01
 -3.04934941e-03 -1.82165168e-02  1.61965322e-02  1.45853609e-02
  8.97004232e-02  2.11753529e-02 -1.13322595e-02  1.37898307e-02
 -1.44742606e-02  5.83038069e-02 -9.02313814e-02 -7.45823458e-02
  1.09646402e-01  2.36695036e-02 -2.57688332e-02  8.15490484e-02
  1.42576511e-03 -1.82034336e-02 -1.73426867e-01 -3.17930132e-02
 -7.68856108e-02  8.63788500e-02  1.19000718e-01  5.18518779e-03
 -2.13312916e-02  4.75826487e-02  3.57612483e-02 -1.77896973e-02
  7.05813617e-02 -6.07864633e-02  2.48750150e-02 -5.70905991e-02
  1.31985303e-02 -8.18155780e-02 -1.96225286e-01  8.16287473e-02
  1.14382379e-01  3.10905278e-02 -5.76074310e-02  1.15664303e-02
  5.26937805e-02  6.90283

In [8]:
# msg_test
# msg_test = msg_test.tolist()
# msg_test = [x for x in msg_test if str(x) != 'nan']
# len(msg_test)
# print("a"+str(msg_test[18])+"b")


In [ ]:
# X_test=[]
# for text in tqdm.tqdm(msg_test):
#     X_test.append(embed(text))
# X_test_new=[]
# for x in X_test:
#     np_tensor = x.numpy()
#     tf_tensor = tf.convert_to_tensor(np_tensor)
#     X_test_new.append(tf_tensor)

X_test=[]
for text in tqdm.tqdm(msg_test):
    X_test.append(embed_bert_cls(str(text), model, tokenizer))
# X_test_new=[]
# for x in X_test:
# #     np_tensor = x.numpy()
#     tf_tensor = tf.convert_to_tensor(x)
#     X_test_new.append(tf_tensor)



  6%|▋         | 226/3597 [00:09<03:11, 17.62it/s]

In [ ]:
X_test_new=[]
for x in X_test:
#     np_tensor = x.numpy()
    tf_tensor = tf.convert_to_tensor(x)
    X_test_new.append(tf_tensor)

In [ ]:
# msg_train
# msg_train = msg_train.tolist()
# msg_train = [x for x in msg_train if str(x) != 'nan']
# len(msg_train)

In [ ]:
# X_train=[]
# for text in tqdm.tqdm(msg_train):
#     X_train.append(embed(text))
# X_train_new=[]
# for x in X_train:
#     np_tensor = x.numpy()
#     tf_tensor = tf.convert_to_tensor(np_tensor)
#     X_train_new.append(tf_tensor)    


In [ ]:

X_train=[]
for text in tqdm.tqdm(msg_train):
    X_train.append(embed_bert_cls(str(text), model, tokenizer))
X_train_new=[]
for x in X_train:
#     np_tensor = x.numpy()
    tf_tensor = tf.convert_to_tensor(x)
    X_train_new.append(tf_tensor)    


In [ ]:
# len(X_train)


In [ ]:
# X_test = embed(msg_test)
# X_test.shape
# # we don't have enough memory to apply embeddings in one shot,
# # so we have to split the data into batches and concatenate them later
# splits = np.array_split(msg_train, 5)
# l = list()
# for split in splits:
#     l.append(embed(split))
# X_train = tf.concat(l, axis=0)
# del l
# X_train.shape
from sklearn.svm import SVC
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
class_weight = compute_class_weight(
    class_weight='balanced', classes=['IT-компании  ',
                                      'Разработка веб-сайтов * ',
                                      'Компьютерное железо  ',
                                      'Настройка Linux * ',
                                      'Гаджеты  ',
                                      'Информационная безопасность * ',
                                      'Социальные сети и сообщества  '], y=y_train) #y_train)
class_weight
# initialize the model and assign weights to each class
clf = SVC(class_weight={"IT-компании  ":class_weight[0], "Разработка веб-сайтов * ":class_weight[1],
                        "Компьютерное железо  ":class_weight[2], "Настройка Linux * ":class_weight[3],
                        "Гаджеты  ":class_weight[4],  "Информационная безопасность * ":class_weight[5],
                        "Социальные сети и сообщества  ":class_weight[6]})
# train the model
clf.fit(X_train_new, y_train) #X_train,y_train
# use the model to predict the testing instances
y_pred = clf.predict(np.array(X_test_new))
# generate the classification report
print(classification_report(y_test, y_pred))
# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(class_weight={"Evidence":class_weight[0], "Concluding Statement":class_weight[1],
#                         "Position":class_weight[2], "Claim":class_weight[3],
#                         "Counterclaim":class_weight[4],  "Lead":class_weight[5],
#                         "Rebuttal":class_weight[6]})
# clf.fit(X_train, y_train)
# y_pred = clf.predict(np.array(X_test))
# print(classification_report(y_test, y_pred))

# from sklearn.naive_bayes import MultinomialNB
# from sklearn.utils import class_weight
# from sklearn.preprocessing import *

# scaler = MinMaxScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# sample = class_weight.compute_sample_weight('balanced', y_train)

# #Classifier Naive Bayes
# naive = MultinomialNB()
# naive.fit(X_train,y_train, sample_weight=sample)
# predictions_NB = naive.predict(X_test)
# print(classification_report(y_test, y_pred))
# num_words = 10000
# max_news_len = 30
# nb_classes=7
# # model_lstm = Sequential()
# # model_lstm.add(Embedding(num_words,32,input_length=max_news_len))
# # model_lstm.add(LSTM(16))
# # model_lstm.add(Dense(7,activation='softmax'))
# # model_lstm.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
# # model_lstm.summary()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(class_weight={"IT-компании  ":class_weight[0], "Разработка веб-сайтов * ":class_weight[1],
                        "Компьютерное железо  ":class_weight[2], "Настройка Linux * ":class_weight[3],
                        "Гаджеты  ":class_weight[4],  "Информационная безопасность * ":class_weight[5],
                        "Социальные сети и сообщества  ":class_weight[6]})
clf.fit(X_train_new, y_train)
y_pred = clf.predict(np.array(X_test_new))
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.utils import class_weight
from sklearn.preprocessing import *

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_new)
X_test = scaler.transform(X_test_new)

sample = class_weight.compute_sample_weight('balanced', y_train)

#Classifier Naive Bayes
naive = MultinomialNB()
naive.fit(X_train,y_train, sample_weight=sample)
predictions_NB = naive.predict(X_test)
print(classification_report(y_test, y_pred))

In [2]:
df = pd.read_csv("qwe.csv")
df = df.applymap(str)
df['tag'] = df['tag'].map({"IT-компании  ": 0, "Разработка веб-сайтов * ": 1,
                        "Компьютерное железо  ": 2, "Настройка Linux * ": 3,
                        "Гаджеты  ": 4,  "Информационная безопасность * ": 5,
                        "Социальные сети и сообщества  ": 6})
df.head(4)
num_classes=7
y = tf.keras.utils.to_categorical(df["tag"].values, num_classes=7)
X_train, X_test, y_train, y_test = train_test_split(df.brave_new_textpunc_stopwords, y,random_state=1)

In [3]:
preprocessor = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
encoder = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-base/1")

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# preprocessor = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
# encoder = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-base/1")
i = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
x = preprocessor(i)
x = encoder(x)
x = tf.keras.layers.Dropout(0.2, name="dropout")(x['pooled_output'])
x = tf.keras.layers.Dense(num_classes, activation='softmax', name="output")(x)
model = tf.keras.Model(i, x)
n_epochs = 1
METRICS = [tf.keras.metrics.CategoricalAccuracy(name="accuracy")]
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", 
                                                      patience = 3,
                                                      restore_best_weights = True)
model.compile(optimizer = "adam",
              loss = "categorical_crossentropy",
              metrics = METRICS)


PATH_TO_MODELS = 'C:\Program Files\modelb\model_0epochs.pb'
for epoch in range(n_epochs):
    model.fit(X_train, 
                      y_train, 
                      epochs = 1,
                      validation_data = (X_test, y_test),
                      callbacks = [earlystop_callback])
    save_name = 'model_%sepochs.pb' % str(epoch)
    model.save(os.path.join(PATH_TO_MODELS, save_name))

NameError: name 'preprocessor' is not defined

In [ ]:
# df1.to_csv('qwe.csv')

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from sklearn.model_selection import train_test_split
import os
n_epochs = 20

model = tf.keras.models.load_model('C:\Program Files\modelb\model_17epochs.h5',custom_objects={'KerasLayer':hub.KerasLayer})
for epoch in range(n_epochs):
    model.fit(X_train, 
                      y_train, 
                      epochs = 1,
                      validation_data = (X_test, y_test)
                      )
    save_name = 'model_%sepochs.h5' % str(epoch)
    model.save(os.path.join('C:\Program Files\modelb', save_name))

 71/331 [=====>........................] - ETA: 15:55 - loss: 0.6111 - accuracy: 0.7830

In [ ]:
X_train.head(4)

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [ ]:
ar  = embed_bert_cls('привет мир', model, tokenizer)

In [ ]:
tensor = tf.convert_to_tensor(ar)

In [ ]:
outputs = embed_bert_cls(str(text_input), model, tokenizer)
tensor = tf.convert_to_tensor(outputs)
# tf.reshape(tensor,[1,312])
tensor =  tf.expand_dims(tensor, axis=0)
tensor =  tf.expand_dims(tensor, axis=0)

